##  🍭 [Day 13](https://adventofcode.com/2018/day/13)

In [0]:
import numpy as np

def parse_char(c):
  """Parse a character and return the direction and orientation 
    when encountering this track. 
    Direction is + 1 for ^ and v, -1 for < and >
    Orientation is + 1 for ^ and >, -1 for v and <"""
  if c == '|':
    return (1, 1, ord('|'))
  elif c == '-':
    return (1, 1, ord('-'))
  elif c == '/':
    return (-1, 1, ord('/'))
  elif c == '\\':
    return (-1, -1, ord('\\'))
  elif c in ['>', '<', '^', 'v']:
    return (1 if c in ['v', '^'] else -1, 1 if c in ['^', '>'] else -1,  -1)
  elif c == '+':
    return (-2, 0, ord('+'))
  else:
    return (0, 0, ord(' '))
  
def parse_inputs(inputs):
  """Parse inputs into a grid and initial cars positions"""
  grid = np.array([[parse_char(c) for c in line] for line in inputs])
  cars = []
  for x, y in zip(*np.where(grid[:, :, -1] < 0)):
    pos = np.array([x, y])
    direction = grid[x, y, 0]
    orientation = grid[x, y, 1]
    grid[x, y] = parse_char('|') if inputs[x][y] in ['^', 'v'] else parse_char('-')
    cars.append((pos, direction, orientation, 0, True)) # (position, direction, orientation, next_turn, alive or crashed)
  return cars, grid
  
def get_character(direction, orientation):
  """Display a car given its current direction and orientation"""
  if direction == 1:
    return ord('^' if orientation == +1 else 'v')
  else:
    return ord('>' if orientation == +1 else '<')
  
def print_grid(grid, cars):
  """Display a grid with cars"""
  g  = np.array(grid)[:, :, -1]
  for pos, direction, orientation, _, alive in cars:
    if alive:
      g[pos[0], pos[1]] = get_character(direction, orientation)
    else:
      g[pos[0], pos[1]] = ord('X')
  for line in g:
    print('\r' * g.shape[0], ''.join(list(map(chr, line))))

In [0]:
def drive(inputs, num_steps=15, part1=True, verbose=False):
  """Let the carts drive for the given number of steps and stop using 
     the rule for the Part 1 problem or Part 2 problem"""
  cars, grid = parse_inputs(inputs)  
  for s in range(num_steps):
    # Sort and filter current cars
    if verbose:
      print_grid(grid, cars)
    cars = sorted(cars, key=lambda x: tuple(x[0]))
    cars = [c for c in cars if c[-1]]
    if not part1:
      print('\rCurrent number of cars: %d' % len(cars), end='')
      
    # Advance each car
    for i, car in enumerate(cars):
      pos, direction, orientation, next_turn, alive = car      
      if not alive: continue
      
      # Crash detection
      dead = []
      for j,  (p, _, _, _, j_alive) in enumerate(cars):
        if i == j or not j_alive:
          continue
        if np.all(pos == p):
          dead.append(j)
          # Part1: First crash detection
          if part1:
            if verbose:
              cars[i] = cars[i][:-1] + [False]
              cars[j] = cars[j][:-1] + [False]
              print_grid(grid, cars)
            return (s, (pos[1], pos[0]))
      # Part2: mark cars as crashed to be removed in the next iteration
      # Stop if there's only one car remaining
      if len(dead):
        for j in (dead + [i]):
          cars[j] = cars[j][:-1] + [False]
        if len(cars) - len(dead) - 1 == 1:
          cars = [c for c in cars if c[-1]]
          if verbose:  print_grid(grid, cars)
          return (s, (cars[0][0][1], cars[0][0][0]))
      else:
        # If the car is still alive, move it
        # Change orientation and direction (if needed) and then go straight
        if grid[pos[0], pos[1], -1] != ord('+'): # normal tracks:
          direction *= grid[pos[0], pos[1], 0]
          orientation *= grid[pos[0], pos[1], 1]
        else:                            # + tracks
          if next_turn == 0:       
            if direction == 1:
              orientation *= -1
            direction *= -1
          elif next_turn == 2:       
            if direction ==  -1:
              orientation *= -1
            direction *= -1
          next_turn = (next_turn + 1) % 3          

        # Update the position
        pos += np.array([(direction > 0) * (- orientation), (direction < 0) * orientation])
        cars[i] = [pos, direction, orientation, next_turn, alive]

In [3]:
%%time
with open("day13.txt", 'r') as f:  
  inputs = f.read().splitlines()
  
print('Time step and position of the first car crash:', drive(inputs, num_steps=200, part1=True, verbose=False))
print()
print('\nTime step of the last crach and position of the survivor:', drive(inputs, num_steps=200000, part1=False, verbose=False))

Time step and position of the first car crash: (123, (143, 43))

Current number of cars: 3
Time step of the last crach and position of the survivor: (22879, (116, 125))
CPU times: user 7.79 s, sys: 1.76 s, total: 9.55 s
Wall time: 10.4 s


In [4]:
#@title Visualization on a toy example
test_inputs = """/->-\\        
|   |  /----\\
| /-+--+-\  |
| | |  | v  |
\\-+-/  \\-+--/
  \\------/   """
test_inputs = test_inputs.splitlines()
print('Time step and position of the first car crash:', drive(test_inputs, num_steps=15, part1=True, verbose=True))

 /->-\        
 |   |  /----\
 | /-+--+-\  |
 | | |  | v  |
 \-+-/  \-+--/
   \------/   
 /-->\        
 |   |  /----\
 | /-+--+-\  |
 | | |  | |  |
 \-+-/  \-v--/
   \------/   
 /--->        
 |   |  /----\
 | /-+--+-\  |
 | | |  | |  |
 \-+-/  \-+>-/
   \------/   
 /---\        
 |   v  /----\
 | /-+--+-\  |
 | | |  | |  |
 \-+-/  \-+->/
   \------/   
 /---\        
 |   |  /----\
 | /-v--+-\  |
 | | |  | |  |
 \-+-/  \-+-->
   \------/   
 /---\        
 |   |  /----\
 | /-+>-+-\  |
 | | |  | |  ^
 \-+-/  \-+--/
   \------/   
 /---\        
 |   |  /----\
 | /-+->+-\  ^
 | | |  | |  |
 \-+-/  \-+--/
   \------/   
 /---\        
 |   |  /----^
 | /-+-->-\  |
 | | |  | |  |
 \-+-/  \-+--/
   \------/   
 /---\        
 |   |  /---<\
 | /-+--+>\  |
 | | |  | |  |
 \-+-/  \-+--/
   \------/   
 /---\        
 |   |  /--<-\
 | /-+--+->  |
 | | |  | |  |
 \-+-/  \-+--/
   \------/   
 /---\        
 |   |  /-<--\
 | /-+--+-\  |
 | | |  | v  |
 \-+-/  \-+--/
   \------/   
 /---\    